In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tabulate import tabulate
from scipy.stats import norm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2

C:\Users\valen\AppData\Local\Temp\ipykernel_14020\3523164832.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_sample = pd.read_csv('sample_submission.csv')

In [4]:
def detecter_groupe(data_test):
    data_test['Group'] = data_test['PassengerId'].str[:4]
    occurences_groupes = data_test['Group'].value_counts()
    data_test['DansGroupe'] = data_test['Group'].map(occurences_groupes) > 1


detecter_groupe(data_test)
    
    
def remplir_colonne_par_groupe(data_test, colonne):
    modif_grp = 0
    
    for groupe in data_test['Group'].unique():
        groupe_data = data_test[data_test['Group'] == groupe]
        groupe_non_nul = groupe_data.dropna(subset=[colonne])     
        valeur_frequente = groupe_non_nul[colonne].mode()
        
        if not valeur_frequente.empty:
            valeur_frequente = valeur_frequente.iloc[0]
            groupe_nan = groupe_data[groupe_data[colonne].isnull()]
            data_test.loc[groupe_nan.index, colonne] = valeur_frequente
            modif_grp += len(groupe_nan)
    
    return modif_grp


def remplacer_valeurs(data_test, colonnes):
    modifs = {}
    for colonne in colonnes:
        print(f"Nombre de case vide dans {colonne} avant 1ere modif: {data_test[colonne].isnull().sum()}")
        modifs[colonne] = remplir_colonne_par_groupe(data_test, colonne)
        #print(f"Nombre de cases modifiées dans {colonne} des personnes avec groupe: {modifs[colonne]}")
    
    for colonne in colonnes:
        print(f"Nombre de case vide dans {colonne} avant 2e modif: {data_test[colonne].isnull().sum()}")
        valeur_frequente = data_test[colonne].mode()[0]
        data_test[colonne] = data_test[colonne].fillna(valeur_frequente)
        nb_modifications_apres = data_test[colonne].isnull().sum()
        print(f"Nombre de case vide dans {colonne} apres 2e modif: {data_test[colonne].isnull().sum()}")
        #print(f"Nombre de cases modifiées dans {colonne} des personnes sans groupe: {modifs[colonne] - nb_modifications_apres}")


colonnes_a_modifier = ["HomePlanet", "Destination", "VIP"]
remplacer_valeurs(data_test, colonnes_a_modifier)


def remplacer_valeurs_cryosleep(data_test):
    valeur = data_test["CryoSleep"].mode()[0]
    nb_modifications = data_test["CryoSleep"].isnull().sum()
    data_test["CryoSleep"] = data_test["CryoSleep"].fillna(valeur)
    nb_modifications_apres = data_test["CryoSleep"].isnull().sum()
    print(f"Nombre de cases modifiées dans CryoSleep: {nb_modifications - nb_modifications_apres}")

def remplacer_valeurs_manquantes_age(data_test):
    moyenne_age = data_test['Age'].mean()
    nb_modifications = data_test['Age'].isnull().sum()
    data_test['Age'] = data_test['Age'].fillna(moyenne_age)
    nb_modifications_apres = data_test['Age'].isnull().sum()
    print(f"Nombre de cases modifiées dans Age: {nb_modifications - nb_modifications_apres}")


def remplacer_valeurs_depense(data_test):
    colonnes = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
    moyennes = data_test[colonnes].mean()
    age_condition = data_test['Age'] >= 18

    for colonne in colonnes:
        nb_modifications = data_test[colonne].isnull().sum()
        data_test[colonne] = data_test[colonne].fillna(
            data_test.apply(lambda row: moyennes[colonne] if row['Age'] >= 12 else 0, axis=1)
        )
        nb_modifications_apres = data_test[colonne].isnull().sum()
        print(f"Nombre de cases modifiées dans {colonne}: {nb_modifications - nb_modifications_apres}")


remplacer_valeurs_cryosleep(data_test)
remplacer_valeurs_depense(data_test)
remplacer_valeurs_manquantes_age(data_test)
data_test.drop(columns=['Name'], inplace=True)
data_test.drop(columns=['Group'], inplace=True)
print(data_test.isnull().sum())


Nombre de case vide dans HomePlanet avant 1ere modif: 87
Nombre de case vide dans Destination avant 1ere modif: 92
Nombre de case vide dans VIP avant 1ere modif: 93
Nombre de case vide dans HomePlanet avant 2e modif: 46
Nombre de case vide dans HomePlanet apres 2e modif: 0
Nombre de case vide dans Destination avant 2e modif: 51
Nombre de case vide dans Destination apres 2e modif: 0
Nombre de case vide dans VIP avant 2e modif: 56
Nombre de case vide dans VIP apres 2e modif: 0
Nombre de cases modifiées dans CryoSleep: 93
Nombre de cases modifiées dans RoomService: 82
Nombre de cases modifiées dans FoodCourt: 106
Nombre de cases modifiées dans ShoppingMall: 98
Nombre de cases modifiées dans Spa: 101
Nombre de cases modifiées dans VRDeck: 80
Nombre de cases modifiées dans Age: 91
PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           100
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa          

C:\Users\valen\AppData\Local\Temp\ipykernel_14020\457983333.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_test[colonne] = data_test[colonne].fillna(valeur_frequente)
C:\Users\valen\AppData\Local\Temp\ipykernel_14020\457983333.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_test["CryoSleep"] = data_test["CryoSleep"].fillna(valeur)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

data_num_stand = scaler.fit_transform(data_test[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]])
data_num_stand = pd.DataFrame(data_num_stand, columns=["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"])

colonnes = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
for colonne in colonnes:
    data_test[colonne] = data_num_stand[colonne] 
    



      RoomService  FoodCourt  ShoppingMall       Spa    VRDeck
0       -0.364143  -0.290713     -0.319230 -0.273958 -0.250651
1       -0.364143  -0.284747     -0.319230  2.283405 -0.250651
2       -0.364143  -0.290713     -0.319230 -0.273958 -0.250651
3       -0.364143   4.118769     -0.319230 -0.109990  0.222931
4       -0.347508  -0.290713      0.826265 -0.273958 -0.250651
...           ...        ...           ...       ...       ...
4272    -0.364143  -0.290713     -0.319230 -0.273958 -0.250651
4273    -0.364143   0.270747     -0.288564 -0.264899 -0.134077
4274    -0.364143  -0.290713     -0.319230 -0.273958 -0.250651
4275    -0.364143   1.485807     -0.319230 -0.273958  0.172740
4276    -0.364143  -0.290713     -0.319230 -0.273958 -0.250651

[4277 rows x 5 columns]


In [6]:
data_test[['Deck', 'Num', 'Side']] = data_test['Cabin'].str.extract(r'([A-G])/(\d+)/([PS])')

deck_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
data_test['Deck'] = data_test['Deck'].map(deck_mapping)

data_test['Side'] = data_test['Side'].map({'P': 0, 'S': 1})

data_test['Deck'] = pd.to_numeric(data_test['Deck'])
data_test['Num'] = pd.to_numeric(data_test['Num'])



In [7]:
def remplacer_valeurs_manquantes(row):
    if pd.isnull(row['Deck']):
        if row['HomePlanet'] == 'Europa':
            return np.random.choice([2.0, 3.0])
        elif row['HomePlanet'] == 'Mars':
            return np.random.choice([4.0, 5.0, 6.0])
        elif row['HomePlanet'] == 'Earth':
            return np.random.choice([6.0, 7.0])
    else:
        return row['Deck']

# Appliquer la fonction pour remplacer les valeurs manquantes dans la colonne "Deck"
data_test['Deck'] = data_test.apply(remplacer_valeurs_manquantes, axis=1)

data_test['Side'].fillna(data_test['Side'].mode()[0], inplace=True)

print(data_test.isnull().sum())


PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           100
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
DansGroupe        0
Deck              0
Num             106
Side              0
dtype: int64


C:\Users\valen\AppData\Local\Temp\ipykernel_14020\902203301.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_test['Side'].fillna(data_test['Side'].mode()[0], inplace=True)


In [8]:
homeplanet_dummies = pd.get_dummies(data_test["HomePlanet"], prefix="HomePlanet")
destination_dummies = pd.get_dummies(data_test["Destination"], prefix="Destination")

data_test = pd.concat([data_test, homeplanet_dummies, destination_dummies], axis=1)

data_test.drop(columns=['Destination'], inplace=True)
data_test.drop(columns=['HomePlanet'], inplace=True)
print(data_test)


     PassengerId  CryoSleep     Cabin        Age    VIP  RoomService  \
0        0013_01       True     G/3/S  27.000000  False    -0.364143   
1        0018_01      False     F/4/S  19.000000  False    -0.364143   
2        0019_01       True     C/0/S  31.000000  False    -0.364143   
3        0021_01      False     C/1/S  38.000000  False    -0.364143   
4        0023_01      False     F/5/S  20.000000  False    -0.347508   
...          ...        ...       ...        ...    ...          ...   
4272     9266_02       True  G/1496/S  34.000000  False    -0.364143   
4273     9269_01      False       NaN  42.000000  False    -0.364143   
4274     9271_01       True   D/296/P  28.658146  False    -0.364143   
4275     9273_01      False   D/297/P  28.658146  False    -0.364143   
4276     9277_01       True  G/1498/S  43.000000  False    -0.364143   

      FoodCourt  ShoppingMall       Spa    VRDeck  DansGroupe  Deck     Num  \
0     -0.290713     -0.319230 -0.273958 -0.250651       

In [9]:
# deck_dummies = pd.get_dummies(data_test_tr['Num'], prefix='Num')
side_dummies = pd.get_dummies(data_test['Side'], prefix='Side')
deck_dummies = pd.get_dummies(data_test['Deck'], prefix='Deck')

# Concaténer les nouvelles variables binaires à votre DataFrame original
data_test = pd.concat([data_test, deck_dummies, side_dummies], axis=1)

# Supprimer la colonne 'Cabin' originale si nécessaire
data_test.drop('Deck', axis=1, inplace=True)
data_test.drop('Num', axis=1, inplace=True)
data_test.drop('Side', axis=1, inplace=True)
data_test.drop('Cabin', axis=1, inplace=True)

print(data_test.head())
data_test.to_csv('data_test_tr.csv', index=False)
data_test = pd.read_csv('data_test_tr.csv')


  PassengerId  CryoSleep   Age    VIP  RoomService  FoodCourt  ShoppingMall  \
0     0013_01       True  27.0  False    -0.364143  -0.290713     -0.319230   
1     0018_01      False  19.0  False    -0.364143  -0.284747     -0.319230   
2     0019_01       True  31.0  False    -0.364143  -0.290713     -0.319230   
3     0021_01      False  38.0  False    -0.364143   4.118769     -0.319230   
4     0023_01      False  20.0  False    -0.347508  -0.290713      0.826265   

        Spa    VRDeck  DansGroupe  ...  Destination_TRAPPIST-1e  Deck_1.0  \
0 -0.273958 -0.250651       False  ...                     True     False   
1  2.283405 -0.250651       False  ...                     True     False   
2 -0.273958 -0.250651       False  ...                    False     False   
3 -0.109990  0.222931       False  ...                     True     False   
4 -0.273958 -0.250651       False  ...                     True     False   

   Deck_2.0  Deck_3.0  Deck_4.0  Deck_5.0  Deck_6.0  Deck_7.0 

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np



categorielle = ['CryoSleep', 'VIP', 'DansGroupe', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Deck_1.0', 'Deck_2.0', 'Deck_3.0', 'Deck_4.0', 'Deck_5.0', 'Deck_6.0', 'Deck_7.0','Side_0.0', 'Side_1.0']
labelEncoder(data_test, categorielle)

X = data_test.drop('Transported', axis=1)
y = data_test['Transported']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
